# Parsing bodyshops map data

In this notebook we import the [repar shops info](https://www.google.com/maps/d/u/2/viewer?hl=en&ll=-23.559310427829978%2C-46.558040275696094&z=10&mid=1T5GSP5wYUiKYY4_VLIxNI086eKYZ8q1v) and parse the information into a table

---
## 1. Parse the data


In [1]:
from pykml import parser
from zipfile import ZipFile
import re, os, psycopg2
import pandas as pd
import numpy as np
from tqdm import tqdm


Load the file

In [2]:
# Local filename
filename = 'Justos Bodyshops.kmz'

# Unzip and load the encoded kml
kmz = ZipFile(filename, 'r')
kml = kmz.open('doc.kml', 'r').read()

# Parse the file
root = parser.fromstring(kml).Document
root.descendantpaths()


['{http://www.opengis.net/kml/2.2}Document',
 '{http://www.opengis.net/kml/2.2}Document.name',
 '{http://www.opengis.net/kml/2.2}Document.description',
 '{http://www.opengis.net/kml/2.2}Document.Style',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle.scale',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle.Icon',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle.Icon.href',
 '{http://www.opengis.net/kml/2.2}Document.Style.LabelStyle',
 '{http://www.opengis.net/kml/2.2}Document.Style.LabelStyle.scale',
 '{http://www.opengis.net/kml/2.2}Document.Style[1]',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle.scale',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle.Icon',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle.Icon.href',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].LabelStyle',
 '{htt

Parse the data

In [3]:
# List of items
all_items = []

# For each entity type
for document in root.Folder[:]:
    
    # Collect the entity type
    entity_type = str(document.name)
    
    # For each placemark
    for placemark in document.Placemark[:]:
        
        placemark_name = str(placemark.name)
        
        # Try to obtain the coordinates
        try:
            placemark_coords = [float (x) for x in re.sub('[^0-9,.]', "", str(placemark.Point.coordinates)).split(',')]
        except:
            placemark_coords = ''
            
        # Try to obtain the details
        try:
            location_details = {y[0]:y[1] for y in [x.split(':') for x in str(placemark.description).split('<br>')]}
        except:
            location_details = {}
            
        # Create new entry
        new_item = {'entity_type': entity_type,
                    'placemark_name': placemark_name,
                    'placemark_coords': placemark_coords}
        for k,v in location_details.items():
            new_item[k]=v if str(v).replace(' ','')!='' else np.nan
            
        # Add to list
        new_item = all_items.append(new_item)
        
        
# Create dataframe
all_items = pd.DataFrame(all_items).drop(columns='name')
all_items.replace('', np.nan, inplace=True)
all_items.head()



,entity_type,placemark_name,placemark_coords,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término),"AV. BANDEIRANTES, 523 VILA OLIMPIA CEP","R MAJ DIOGO, 590 BELA VISTA CEP"
0,Cities,Arujá,"[46.3227109, 23.3951365, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cities,PIN 01 - BEXIGA,"[46.6395571, 23.5557714, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cities,Barueri,"[46.8785468, 23.5035117, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cities,Biritiba-Mirim,"[46.0409059, 23.5704516, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cities,Caieiras,"[46.7466172, 23.362065, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for col in all_items.columns[-2:]:
    display(all_items[all_items[col].notna()])
    
all_items.drop(columns=all_items.columns[-2:], inplace=True)

,entity_type,placemark_name,placemark_coords,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término),"AV. BANDEIRANTES, 523 VILA OLIMPIA CEP","R MAJ DIOGO, 590 BELA VISTA CEP"
141,12 Pins,PRO SERVICE SERVIÇOS - Itaim,"[46.6777467, 23.5841772, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4553010 SAO PAULO - SP,NaN


,entity_type,placemark_name,placemark_coords,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término),"AV. BANDEIRANTES, 523 VILA OLIMPIA CEP","R MAJ DIOGO, 590 BELA VISTA CEP"
142,12 Pins,RESTAUCAR - Paraiso,"[46.6427902, 23.57545, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01324000 SAO PAULO - SP


---
## 2. Sanity checks

Can't access the addresses I do see in Maps for some places, for example `Dencar` in `Repair shops (to be visited)`

In [5]:
print(root.Folder[2].Placemark[0].name)
display(root.Folder[2].Placemark[0].descendantpaths())
display(root.Folder[2].Placemark[0].Point.descendantpaths())

Dencar


['{http://www.opengis.net/kml/2.2}Placemark',
 '{http://www.opengis.net/kml/2.2}Placemark.name',
 '{http://www.opengis.net/kml/2.2}Placemark.styleUrl',
 '{http://www.opengis.net/kml/2.2}Placemark.Point',
 '{http://www.opengis.net/kml/2.2}Placemark.Point.coordinates']

['{http://www.opengis.net/kml/2.2}Point',
 '{http://www.opengis.net/kml/2.2}Point.coordinates']

Count by entity type in the map

In [6]:
all_items.entity_type.value_counts()

Cities                             43
Contrato assinado                  40
BUSCAR                             22
Repair shops (to be visited)       21
12 Pins                            16
Repair shops (waiting sign-off)     2
Name: entity_type, dtype: int64

There's some bodyshops with no description!

In [7]:
bodyshops = all_items[all_items['entity_type']=='Contrato assinado'].reset_index().drop(columns='index')
bodyshops.isna().sum()

entity_type                          0
placemark_name                       3
placemark_coords                    37
Razão Social                         3
CNPJ                                 3
Telefone                             3
Responsável                          3
Logradouro                           3
Número                               3
Complemento                         39
Bairro                               3
Cidade                               3
Estado                               3
CEP                                  3
Horário de recebimento (início)      3
Horário de recebimento (término)     3
dtype: int64

There's indeed 3 bodyshops with no name or descriptions

In [8]:
[x.name for x in root.Folder[4].Placemark[:]]

['Scattini Funilaria',
 'Oficina Marques (Vila Mascote)',
 'Oficina Marques (Santo Amaro)',
 'Ponce e Filho',
 'Dimas Norte',
 'Coronato',
 'Evolution',
 'Quatrocentos',
 'Dikar',
 'New Fenix',
 'Volens Reparações',
 'Goias Car (São Caetano)',
 'Goias Car (Santo Andre)',
 'Solução Automotiva',
 'Personal Garage',
 'Quadro Rodas Mais',
 'Torque Tamboré',
 'Solicar',
 'Centro Automotivo GL',
 'Funilaria Paulista',
 'Corifeu Automec',
 'Celio´s Car Taboão',
 'CB Car',
 'Nicola Garage',
 'Seadriccar',
 'Jotecar',
 'EcoFix ',
 'MsFix',
 'Auto Palace',
 'Club  Car',
 'Flama Car',
 'Volare Reparos',
 'ArtSystens',
 'Donicar',
 'Ribeiro Renovadora',
 'Studio Marilia Botelho',
 'Office Car',
 '',
 '',
 '']

In [9]:
[x.description for x in root.Folder[4].Placemark[:]][-3:]

['Razão Social: <br>CNPJ: <br>Telefone: <br>Responsável: <br>Logradouro: <br>Número: <br>Complemento: <br>Bairro: <br>Cidade: <br>Estado: <br>CEP: <br>Horário de recebimento (início): <br>Horário de recebimento (término): <br>name: Bonneville',
 'Razão Social: <br>CNPJ: <br>Telefone: <br>Responsável: <br>Logradouro: <br>Número: <br>Complemento: <br>Bairro: <br>Cidade: <br>Estado: <br>CEP: <br>Horário de recebimento (início): <br>Horário de recebimento (término): <br>name: Car Solution Express',
 'Razão Social: <br>CNPJ: <br>Telefone: <br>Responsável: <br>Logradouro: <br>Número: <br>Complemento: <br>Bairro: <br>Cidade: <br>Estado: <br>CEP: <br>Horário de recebimento (início): <br>Horário de recebimento (término): <br>name: Mec Max']

---
### 3. Assign latitude and longitude to addresses

We work with current bodyshops

In [10]:
bodyshops

,entity_type,placemark_name,placemark_coords,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término)
0,Contrato assinado,Scattini Funilaria,NaN,Scattini Funilaria e Pintura de Autos Ltda,52.644.093/0001-90,(11) 5543-9622,Junior,"Al,. dos Nhambiquaras",1542,NaN,Moema,São Paulo,SP,04090-003,8,6
1,Contrato assinado,Oficina Marques (Vila Mascote),NaN,Oficina Marques Funilaria e Pintura Ltda,07.427.476/0001-52,(11) 5535-1903,Cae,Av. Santa Catarina,2580,NaN,Vila Mascote,São Paulo,SP,04378-200,8,5
2,Contrato assinado,Oficina Marques (Santo Amaro),NaN,Marques e Marques Funilaria e Pintura Ltda,07.427.481/001-65,(11) 5535-1903,Cae,Av. Santo Amaro,4408,NaN,Brooklin,São Paulo,SP,04556-500,8,5
3,Contrato assinado,Ponce e Filho,NaN,Ponce & Filho Ltda,54.824.826/0001-02,(11) 2965-7383,Emilio,R. Butiá,42,NaN,Vila Reg. Feijó,São Paulo,SP,03346-010,8,5
4,Contrato assinado,Dimas Norte,NaN,Fiola Reparadora de Veiciulos Eireli,17.237.668/0001-50,(11) 2904-8400,Fiola,Av. Gen. Ataliba Leonel,1727,NaN,Carandiru,São Paulo,SP,02033-010,8,5
5,Contrato assinado,Coronato,NaN,Coronato Reparação Automotiva Ltda,10.764.089/0001-16,(11) 36110934,Elcio,Av. Santa Marina,1520,NaN,Água Branca,São Paulo,SP,05036-001,8,6
6,Contrato assinado,Evolution,NaN,Evolution Express Funilaria e Pintura Ltda,11.027.040/0001-43,(11) 3225-3200,Silvio,R. dos Italianos,708,NaN,Bom Retiro,São Paulo,SP,01131-000,8,5
7,Contrato assinado,Quatrocentos,NaN,Oficina Quatrocentos Funilaria e Pintura Ltda,45.703.923/0001-48,(11) 5541-3100,Celso,R. dos Inocentes,537,NaN,Socorro,São Paulo,SP,04764-050,7,5
8,Contrato assinado,Dikar,NaN,Auto Nog Peças e Serviços Eireli,06.272.146/0001-72,(11) 4362-9700,Nogueira,R. Marabá,80,NaN,Rudge Ramos,São Bernardo do Campo,SP,09635-040,8,5
9,Contrato assinado,New Fenix,NaN,New Fenix Reparação Automotiva Ltda,14.523.083/0001-08,(11) 4376-4376,Rodrigo Souza,R. Amador Bueno,430,NaN,Piratininga,Osasco,SP,06230-100,8,5


Download table of coords and CEP codes

In [17]:
# Create the geolocator object
from tqdm import tqdm
from geopy.geocoders import MapBox
geolocator = MapBox(api_key=os.environ['MAPBOX_DS_TOKEN']# timeout=time_out, adapter_factory=AioHTTPAdapter,
           )

# Function to modify the endpoint to places-permanent which apparently let's you export the results (I don't need it)
def places_permanent(geolocator):
    geolocator.api_path = "/geocoding/v5/mapbox.places-permanent/{query}.json/"
    geolocator.api = "%s://%s%s" % (
        geolocator.scheme,
        geolocator.domain,
        geolocator.api_path,
    )
    return geolocator

# Create the geolocator object
from geopy.geocoders import MapBox
geolocator = MapBox(api_key=os.environ['MAPBOX_DS_TOKEN']# timeout=time_out, adapter_factory=AioHTTPAdapter,
           )

# Now for each bodyshop
responses = []
for i, row in tqdm(bodyshops.iterrows()):
    
    try:

        # Create the address for the request
        request_address = row['Logradouro'].lstrip() + ", " +\
                          row['Número'].lstrip() + ", " +\
                          row['Cidade'].lstrip() + ", " +\
                          row['Bairro'].lstrip() + " - " +\
                          row['Estado'].lstrip() + ", " +\
                          row['CEP'].lstrip() + ", Brazil"

        # Send the request
        response = geolocator.geocode(request_address,
                                      exactly_one=False,
                                      country='BR')

        # Store the output
        responses.append([{'mapped_address': result.address,
                           'latitude': result.latitude,
                           'longitude': result.longitude} for result in response])

    except:
        responses.append({'mapped_address': np.nan,
                          'latitude': np.nan,
                          'longitude': np.nan})
                      
    

40it [00:05,  6.94it/s]


Review results to assign the correct one - typically the best result shares CEP, but many times this isn't the case... at some point I'd have to automate this

In [63]:
correct_result_index = {
    0: 0,
    1: 0,
    2: 1,
    3: 1,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: -1,
    9: -1,
    10: 0,
    11: 0,
    12: 0,
    13: -1,
    14: -1,
    15: 0,
    16: -1,
    17: 0,
    18: -1,
    19: 3,
    20: 2,
    21: -1,
    22: 0,
    23: -1,
    24: -1,
    25: 2,
    26: -1,
    27: 1,
    28: 0,
    29: 2,
    30: 1,
    31: -1,
    32: -1,
    33: 0,
    34: 0,
    35: 0,
    36: -1,
    37: -1,
    38: -1,
    39: -1
}

Check the cases where none of the options seem to be valid

In [66]:
for k,v in correct_result_index.items():
    if v==-1:
        display(responses[k])
        print()
        for key in ['Logradouro', 'Número', 'Bairro', 'Cidade', 'CEP']:
            print(key)
            print(bodyshops.iloc[k][key])
        print('\n----\n')
    

[{'mapped_address': 'Rudge Ramos, São Bernardo Do Campo - São Paulo, Brazil',
  'latitude': -23.6591,
  'longitude': -46.5683},
 {'mapped_address': 'Hocca Bar, São Bernardo Plaza Shopping, São Bernardo Do Campo, São Paulo 09790, Brazil',
  'latitude': -23.722901,
  'longitude': -46.543064},
 {'mapped_address': 'Cantina do Zelão, R. Jurubatuba, 926, São Bernardo Do Campo, São Paulo 09725, Brazil',
  'latitude': -23.705457,
  'longitude': -46.553238},
 {'mapped_address': 'Giramundo Bar & Ristorante, R. Olegário Herculano, 235, São Bernardo Do Campo, São Paulo 09732, Brazil',
  'latitude': -23.682619,
  'longitude': -46.55846},
 {'mapped_address': 'Padaria Bela Petrópolis, R. Cidade de São Sebastião, 203, São Bernardo Do Campo, São Paulo 09761, Brazil',
  'latitude': -23.710201,
  'longitude': -46.53935}]


Logradouro
 R. Marabá
Número
 80
Bairro
 Rudge Ramos
Cidade
 São Bernardo do Campo
CEP
 09635-040

----



[{'mapped_address': 'Rua Amador Bueno, Piratininga, Osasco - São Paulo, 06236, Brazil',
  'latitude': -23.518342,
  'longitude': -46.794545},
 {'mapped_address': 'Rua Amador Bueno 430, Paquetá, Santos - São Paulo, 11013, Brazil',
  'latitude': -23.937599,
  'longitude': -46.319753},
 {'mapped_address': 'Rua Amador Bueno 430, Itapevi - São Paulo, 06680, Brazil',
  'latitude': -23.531616,
  'longitude': -46.977936},
 {'mapped_address': 'Rua Amador Bueno 430, Nasser, Campo Grande - Mato Grosso do Sul, 70116, Brazil',
  'latitude': -20.425939,
  'longitude': -54.635548},
 {'mapped_address': 'Rua Amador Bueno 430, Tabatinga, Camaragibe - Pernambuco, 54754, Brazil',
  'latitude': -7.991673,
  'longitude': -34.974019}]


Logradouro
 R. Amador Bueno
Número
 430
Bairro
 Piratininga
Cidade
 Osasco
CEP
 06230-100

----



[{'mapped_address': 'Rua Salvador Peluso Basile Taboão Da Serra - São Paulo, 05544, Brazil',
  'latitude': -23.59965,
  'longitude': -46.761256},
 {'mapped_address': 'Taboão Da Serra, São Paulo, Brazil',
  'latitude': -23.6122,
  'longitude': -46.7791},
 {'mapped_address': 'Santa Rosa, Rio Grande do Sul, Brazil',
  'latitude': -27.8644,
  'longitude': -54.4779},
 {'mapped_address': 'Santa Rosa, Niterói - Rio de Janeiro, Brazil',
  'latitude': -22.8992,
  'longitude': -43.0986},
 {'mapped_address': 'Santa Rosa, Cubatão - São Paulo, Brazil',
  'latitude': -23.89,
  'longitude': -46.43}]

...


Logradouro
nan
Número
nan
Bairro
nan
Cidade
nan
CEP
nan

----



{'mapped_address': nan, 'latitude': nan, 'longitude': nan}


Logradouro
nan
Número
nan
Bairro
nan
Cidade
nan
CEP
nan

----



{'mapped_address': nan, 'latitude': nan, 'longitude': nan}


Logradouro
nan
Número
nan
Bairro
nan
Cidade
nan
CEP
nan

----



Add responses to dataset

In [67]:
#bodyshops = pd.concat([bodyshops, pd.DataFrame(responses)], axis=1)
#bodyshops.head()
